# Working with Tigris in Pixeltable

This tutorial demonstrates how to configure Pixeltable to use [Tigris](https://tigrisdata.com) for storage. This lets you store unlimited amounts of images in Tigris' global data plane, allowing your images to load fast everywhere.

## Prerequisites

* A Tigris account, bucket, and access keypair ([https://storage.new](https://storage.new))

## Important notes

* Tigris usage may incur costs based on your plan.
* Be mindful of sensitive data and consider security measures when integrating with external services.

First you need to install required libraries and enter a Tigris access keypair obtained via the Tigris Admin Console.

## Set up environment

First, let's install Pixeltable:

In [ ]:
%pip install -qU pixeltable boto3 datasets

## Configure authentication

These steps will have you enter in your Tigris credentials:

In [ ]:
import os
from getpass import getpass


os.environ["AWS_ACCESS_KEY_ID"] = getpass("Tigris access key ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass("Tigris secret access key")
bucket_name = getpass("Tigris bucket name")

os.environ["AWS_ENDPOINT_URL_S3"] = "https://t3.storage.dev"
os.environ["AWS_REGION"] = "auto"
os.environ["PIXELTABLE_INPUT_MEDIA_DEST"] = f"s3://{bucket_name}/input/"
os.environ["PIXELTABLE_OUTPUT_MEDIA_DEST"] = f"s3://{bucket_name}/output/"

## Create a table for images

Now let's create a table that will contain images from the [XeIaso/botw-screenshots-captioned](https://huggingface.co/datasets/XeIaso/botw-screenshots-captioned) dataset:

In [ ]:
import pixeltable as pxt
from datasets import load_dataset


# Create directory for this demo
pxt.drop_dir("tigris", force=True)
pxt.create_dir("tigris", if_exists="replace")


# Load the dataset
ds = load_dataset("XeIaso/botw-screenshots-captioned")
# Import it to pixeltable with the name screenshots
pxt.drop_table("tigris.screenshots", force=True)
screenshots = pxt.create_table("tigris.screenshots", source=ds, if_exists="replace")

Once the import is done, you can create thumbnails with a [computed column](https://docs.pixeltable.com/tutorials/computed-columns):

In [ ]:
# Add a computed column for thumbnails
# Uses output_media_dest by default, or specify a custom destination
screenshots.add_computed_column(
    thumbnail=screenshots.image.resize((256, 256)),
    destination=f"s3://{bucket_name}/botw-screenshots/thumbnails/"
)

And then inspect that with the `collect` method:

In [ ]:
results = screenshots.limit(1).collect()
results

## Getting URLs for your files

When your files are in object storage, you can get URLs that point directly to them. These URLs work in HTML, APIs, or any application you need to serve media with. Fetch them with the `.fileurl` property:

In [ ]:
screenshots.select(
  image=screenshots.image,
  image_url=screenshots.image.fileurl,
  thumbnail=screenshots.thumbnail,
  thumbnail_url=screenshots.thumbnail.fileurl,
).limit(1).collect()

## Generating Presigned URLs

For private buckets or when you need time-limited access to files, use presigned URLs. These are temporary, authenticated URLs that allow anyone to access your files for a limited time without needing credentials.

Use the `presigned_url` function from `pixeltable.functions.net`:

In [ ]:
from pixeltable.functions import net

# Generate presigned URLs with 1-hour expiration (3600 seconds)
screenshots.select(
    image=screenshots.image,
    image_url=screenshots.image.fileurl,
    image_presigned=net.presigned_url(screenshots.image.fileurl, 3600),
    thumbnail=screenshots.thumbnail,
    thumbnail_url=screenshots.thumbnail.fileurl,
    thumbnail_presigned=net.presigned_url(screenshots.thumbnail.fileurl, 3600),
).limit(1).collect()

### Common expiration times

| Use case | Seconds | Example |
|----------|---------|--------|
| Quick preview | 300 | 5 minutes |
| API response | 3600 | 1 hour |
| Download link | 86400 | 24 hours |
| Long-term share | 604800 | 7 days |

In [ ]:
# Example: Generate a 24-hour presigned URL for sharing
share_urls = screenshots.select(
    share_link=net.presigned_url(screenshots.image.fileurl, 86400),
).limit(3).collect()

## What you learned

* When you configure Pixeltable to use Tigris to store images, adding images transparently uploads them into Tigris for global distribution.
* You can override where images are stored in Tigris using the `destination=` kwarg when creating computed columns.
* Use the `.fileurl` property in queries to get URLs for your stored files.
* Use `net.presigned_url()` to generate time-limited, authenticated URLs for private bucket access.

Pixeltable handles everything else for you.

## Next steps

* See the [Cloud Storage documentation](https://docs.pixeltable.com/integrations/cloud-storage) for complete provider setup and authentication details.
* Check out [Pixeltable Configuration](https://docs.pixeltable.com/platform/configuration) for all config options.
* Join our [Discord community](https://pixeltable.com/discord) if you have questions.

## Additional Resources

* [Pixeltable Documentation](https://docs.pixeltable.com/)
* [Tigris Documentation](https://www.tigrisdata.com/docs/)